In [2]:
import pickle
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.multioutput import MultiOutputRegressor
import pickle

# New Section

# New Section

In [10]:
input_data = pd.read_csv('/content/sample_data/chicago_taxi.csv')
input_data.head(5)

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Extras,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location
0,011106b6114f83af0c17aace3867a464a7fc742b,4628ef9dfa973bdfe877c5aa9d9738f9dc1204e54f2f1a...,12/31/2023 11:45:00 PM,01/01/2024 12:00:00 AM,982.0,0.81,1.703108e+10,1.703108e+10,8.0,8.0,...,1.0,13.25,Credit Card,Chicago Independents,41.893216,-87.637844,POINT (-87.6378442095 41.8932163595),41.892508,-87.626215,POINT (-87.6262149064 41.8925077809)
1,0353da5e93e2f5f973dc685d76fcd15f6bc0256e,ad4b1730fcbfdb84e41313179a688924012db322823f48...,12/31/2023 11:45:00 PM,12/31/2023 11:45:00 PM,12.0,0.57,NaN,NaN,24.0,24.0,...,17.0,21.50,Cash,5 Star Taxi,41.901207,-87.676356,POINT (-87.6763559892 41.9012069941),41.901207,-87.676356,POINT (-87.6763559892 41.9012069941)
2,05729c04e16ca660907a57d29070d249408b7fcc,22b403ba1c4de25cb19049e9c08b6d1339a9eea3ec06f9...,12/31/2023 11:45:00 PM,01/01/2024 12:00:00 AM,883.0,4.28,NaN,NaN,42.0,38.0,...,0.0,13.50,Prcard,Flash Cab,41.778877,-87.594925,POINT (-87.5949254391 41.7788768603),41.812949,-87.617860,POINT (-87.6178596758 41.8129489392)
3,0df5288f95b24b9c8b659d7368516f5a2ccdd63a,d6e1a9e103336c396201abe9ceb00795fcd41e14ccbf54...,12/31/2023 11:45:00 PM,12/31/2023 11:45:00 PM,9.0,0.00,NaN,NaN,32.0,32.0,...,0.0,26.42,Mobile,Flash Cab,41.878866,-87.625192,POINT (-87.6251921424 41.8788655841),41.878866,-87.625192,POINT (-87.6251921424 41.8788655841)
4,0ed132d8a023dde8abb89dba58fa04f35fe9bdb1,80919866bf50fd1163efd329eb8a9fdf198fda2465473a...,12/31/2023 11:45:00 PM,01/01/2024 12:00:00 AM,769.0,0.74,1.703108e+10,1.703108e+10,8.0,8.0,...,0.0,9.00,Mobile,Blue Ribbon Taxi Association,41.893216,-87.637844,POINT (-87.6378442095 41.8932163595),41.892508,-87.626215,POINT (-87.6262149064 41.8925077809)


In [11]:
input_data.count()

,0
Trip ID,546787
Taxi ID,546786
Trip Start Timestamp,546786
Trip End Timestamp,546769
Trip Seconds,546684
Trip Miles,546786
Pickup Census Tract,229582
Dropoff Census Tract,222590
Pickup Community Area,532488
Dropoff Community Area,500166


In [12]:
df = input_data.rename(columns={
    'Trip Start Timestamp': 'trip_start_timestamp',
    'Trip End Timestamp': 'trip_end_timestamp',
    'Trip Seconds': 'trip_seconds',
    'Trip Miles': 'trip_miles',
    'Trip Total': 'trip_total',
    'Pickup Community Area': 'pickup_community_area',
    'Dropoff Community Area': 'dropoff_community_area',
    'Fare':'fare',
    'Tips':'tips',
    'Tolls':'tolls',
    'extras':'Extras',
    'Payment Type': 'payment_type',
    'Pickup Centroid Latitude':'pickup_centroid_latitude',
    'Pickup Centroid Longitude':'pickup_centroid_longitude',
    'Pickup Centroid Location':'pickup_centroid_location',
    'Dropoff Centroid Latitude':'dropoff_centroid_latitude',
    'Dropoff Centroid Longitude':'dropoff_centroid_longitude',
    'Dropoff Centroid  Location':'dropoff_centroid_location'
})
df.head()

,Trip ID,Taxi ID,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,Pickup Census Tract,Dropoff Census Tract,pickup_community_area,dropoff_community_area,...,Extras,trip_total,payment_type,Company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location
0,011106b6114f83af0c17aace3867a464a7fc742b,4628ef9dfa973bdfe877c5aa9d9738f9dc1204e54f2f1a...,12/31/2023 11:45:00 PM,01/01/2024 12:00:00 AM,982.0,0.81,1.703108e+10,1.703108e+10,8.0,8.0,...,1.0,13.25,Credit Card,Chicago Independents,41.893216,-87.637844,POINT (-87.6378442095 41.8932163595),41.892508,-87.626215,POINT (-87.6262149064 41.8925077809)
1,0353da5e93e2f5f973dc685d76fcd15f6bc0256e,ad4b1730fcbfdb84e41313179a688924012db322823f48...,12/31/2023 11:45:00 PM,12/31/2023 11:45:00 PM,12.0,0.57,NaN,NaN,24.0,24.0,...,17.0,21.50,Cash,5 Star Taxi,41.901207,-87.676356,POINT (-87.6763559892 41.9012069941),41.901207,-87.676356,POINT (-87.6763559892 41.9012069941)
2,05729c04e16ca660907a57d29070d249408b7fcc,22b403ba1c4de25cb19049e9c08b6d1339a9eea3ec06f9...,12/31/2023 11:45:00 PM,01/01/2024 12:00:00 AM,883.0,4.28,NaN,NaN,42.0,38.0,...,0.0,13.50,Prcard,Flash Cab,41.778877,-87.594925,POINT (-87.5949254391 41.7788768603),41.812949,-87.617860,POINT (-87.6178596758 41.8129489392)
3,0df5288f95b24b9c8b659d7368516f5a2ccdd63a,d6e1a9e103336c396201abe9ceb00795fcd41e14ccbf54...,12/31/2023 11:45:00 PM,12/31/2023 11:45:00 PM,9.0,0.00,NaN,NaN,32.0,32.0,...,0.0,26.42,Mobile,Flash Cab,41.878866,-87.625192,POINT (-87.6251921424 41.8788655841),41.878866,-87.625192,POINT (-87.6251921424 41.8788655841)
4,0ed132d8a023dde8abb89dba58fa04f35fe9bdb1,80919866bf50fd1163efd329eb8a9fdf198fda2465473a...,12/31/2023 11:45:00 PM,01/01/2024 12:00:00 AM,769.0,0.74,1.703108e+10,1.703108e+10,8.0,8.0,...,0.0,9.00,Mobile,Blue Ribbon Taxi Association,41.893216,-87.637844,POINT (-87.6378442095 41.8932163595),41.892508,-87.626215,POINT (-87.6262149064 41.8925077809)


In [13]:
df['trip_start_timestamp'] = pd.to_datetime(df['trip_start_timestamp'])
df['trip_end_timestamp'] = pd.to_datetime(df['trip_end_timestamp'])
filtered_df = df[
    (df['trip_start_timestamp'] >= '2023-10-01') &
    (df['trip_seconds'] > 60) &
    (df['trip_seconds'] < 6*60*60) &
    (df['trip_miles'] > 0) &
    (df['trip_total'] > 3) &
    (df['pickup_centroid_location'].notnull())
]
filtered_df["trip_hours"] = round(filtered_df["trip_seconds"] / 3600, 2)
filtered_df["trip_speed"] = round(filtered_df["trip_miles"] / filtered_df["trip_hours"], 2)

filtered_df.head()


<ipython-input-13-78afdee46017>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['trip_start_timestamp'] = pd.to_datetime(df['trip_start_timestamp'])
<ipython-input-13-78afdee46017>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['trip_end_timestamp'] = pd.to_datetime(df['trip_end_timestamp'])
<ipython-input-13-78afdee46017>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["trip_hours"] = round(filtered_df["trip_seconds"] / 3600, 2)
<ipython-input-13-7

,Trip ID,Taxi ID,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,Pickup Census Tract,Dropoff Census Tract,pickup_community_area,dropoff_community_area,...,payment_type,Company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location,trip_hours,trip_speed
0,011106b6114f83af0c17aace3867a464a7fc742b,4628ef9dfa973bdfe877c5aa9d9738f9dc1204e54f2f1a...,2023-12-31 23:45:00,2024-01-01 00:00:00,982.0,0.81,1.703108e+10,1.703108e+10,8.0,8.0,...,Credit Card,Chicago Independents,41.893216,-87.637844,POINT (-87.6378442095 41.8932163595),41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),0.27,3.00
2,05729c04e16ca660907a57d29070d249408b7fcc,22b403ba1c4de25cb19049e9c08b6d1339a9eea3ec06f9...,2023-12-31 23:45:00,2024-01-01 00:00:00,883.0,4.28,NaN,NaN,42.0,38.0,...,Prcard,Flash Cab,41.778877,-87.594925,POINT (-87.5949254391 41.7788768603),41.812949,-87.617860,POINT (-87.6178596758 41.8129489392),0.25,17.12
4,0ed132d8a023dde8abb89dba58fa04f35fe9bdb1,80919866bf50fd1163efd329eb8a9fdf198fda2465473a...,2023-12-31 23:45:00,2024-01-01 00:00:00,769.0,0.74,1.703108e+10,1.703108e+10,8.0,8.0,...,Mobile,Blue Ribbon Taxi Association,41.893216,-87.637844,POINT (-87.6378442095 41.8932163595),41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),0.21,3.52
5,1686a96446c079079e6b53574c3d4f78da6fcfdc,64b71bd4e488e9c5571cdfcca045e7cb7a4abb0931f17b...,2023-12-31 23:45:00,2024-01-01 00:00:00,954.0,0.41,1.703108e+10,1.703108e+10,8.0,8.0,...,Cash,Flash Cab,41.893216,-87.637844,POINT (-87.6378442095 41.8932163595),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),0.26,1.58
6,1adedef3b9733f6a1859137ce37d8c685ad36cea,7d2e7cdd59237335e96b9b1a897a5e48cb4df467e6c092...,2023-12-31 23:45:00,2023-12-31 23:45:00,121.0,0.45,NaN,NaN,8.0,8.0,...,Credit Card,City Service,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),41.899602,-87.633308,POINT (-87.6333080367 41.899602111),0.03,15.00


In [14]:
# set constraints to remove outliers
filtered_df = filtered_df[filtered_df["trip_total"] > 3]

filtered_df = filtered_df[(filtered_df["trip_miles"] > 0) & (filtered_df["trip_miles"] < 300)]

filtered_df = filtered_df[filtered_df["trip_seconds"] >= 60]

filtered_df = filtered_df[filtered_df["trip_hours"] <= 2]

filtered_df = filtered_df[filtered_df["trip_speed"] <= 70]
filtered_df = filtered_df[filtered_df["trip_total"] < 3000]
filtered_df["trip_start_timestamp"] = pd.to_datetime(filtered_df["trip_start_timestamp"])
filtered_df["dayofweek"] = filtered_df["trip_start_timestamp"].dt.dayofweek
filtered_df["hour"] = filtered_df["trip_start_timestamp"].dt.hour

# bucket and encode the dayofweek and hour
filtered_df["dayofweek"] = filtered_df["dayofweek"].apply(lambda x: 0 if x in [5, 6] else 1)
filtered_df["hour"] = filtered_df["hour"].apply(lambda x: 0 if x in [23, 0, 1, 2, 3, 4, 5, 6, 7] else 1)

filtered_df.reset_index(drop=True, inplace=True)
filtered_df.head()

<ipython-input-14-4ee2d61f9d84>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["trip_start_timestamp"] = pd.to_datetime(filtered_df["trip_start_timestamp"])
<ipython-input-14-4ee2d61f9d84>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["dayofweek"] = filtered_df["trip_start_timestamp"].dt.dayofweek
<ipython-input-14-4ee2d61f9d84>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

,Trip ID,Taxi ID,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,Pickup Census Tract,Dropoff Census Tract,pickup_community_area,dropoff_community_area,...,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location,trip_hours,trip_speed,dayofweek,hour
0,011106b6114f83af0c17aace3867a464a7fc742b,4628ef9dfa973bdfe877c5aa9d9738f9dc1204e54f2f1a...,2023-12-31 23:45:00,2024-01-01 00:00:00,982.0,0.81,1.703108e+10,1.703108e+10,8.0,8.0,...,41.893216,-87.637844,POINT (-87.6378442095 41.8932163595),41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),0.27,3.00,0,0
1,05729c04e16ca660907a57d29070d249408b7fcc,22b403ba1c4de25cb19049e9c08b6d1339a9eea3ec06f9...,2023-12-31 23:45:00,2024-01-01 00:00:00,883.0,4.28,NaN,NaN,42.0,38.0,...,41.778877,-87.594925,POINT (-87.5949254391 41.7788768603),41.812949,-87.617860,POINT (-87.6178596758 41.8129489392),0.25,17.12,0,0
2,0ed132d8a023dde8abb89dba58fa04f35fe9bdb1,80919866bf50fd1163efd329eb8a9fdf198fda2465473a...,2023-12-31 23:45:00,2024-01-01 00:00:00,769.0,0.74,1.703108e+10,1.703108e+10,8.0,8.0,...,41.893216,-87.637844,POINT (-87.6378442095 41.8932163595),41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),0.21,3.52,0,0
3,1686a96446c079079e6b53574c3d4f78da6fcfdc,64b71bd4e488e9c5571cdfcca045e7cb7a4abb0931f17b...,2023-12-31 23:45:00,2024-01-01 00:00:00,954.0,0.41,1.703108e+10,1.703108e+10,8.0,8.0,...,41.893216,-87.637844,POINT (-87.6378442095 41.8932163595),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),0.26,1.58,0,0
4,1adedef3b9733f6a1859137ce37d8c685ad36cea,7d2e7cdd59237335e96b9b1a897a5e48cb4df467e6c092...,2023-12-31 23:45:00,2023-12-31 23:45:00,121.0,0.45,NaN,NaN,8.0,8.0,...,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),41.899602,-87.633308,POINT (-87.6333080367 41.899602111),0.03,15.00,0,0


In [15]:
# Features we'll use to predict 'trip_total'
feature_cols = [
    'pickup_centroid_latitude', 'pickup_centroid_longitude',
    'dropoff_centroid_latitude', 'dropoff_centroid_longitude',
     'dayofweek', 'hour','trip_miles'
]
X = filtered_df[feature_cols]
y = filtered_df[['fare','trip_seconds']]

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Using Random Forest Regressor - good for tabular data
#model = RandomForestRegressor(n_estimators=100, random_state=42)
model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
model.fit(X_train, y_train)
# Predict on test set
y_pred = model.predict(X_test)

# Evaluation Metrics
print("Root Mean Squared Error:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R^2 Score:", r2_score(y_test, y_pred))


Root Mean Squared Error: 375.1555125551123
R^2 Score: 0.7843538872032564


In [17]:
import pickle
with open('/content/trip_total_predictor.pkl', 'wb') as f:
    pickle.dump(model, f)

print("Model saved as 'trip_total_predictor.pkl'")

Model saved as 'trip_total_predictor.pkl'


In [18]:
import pandas as pd
import numpy as np
import pickle


# Ensure numeric columns
for col in ["trip_seconds", "trip_miles", "fare", "tips", "trip_total"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Derived columns
df["trip_minutes"] = df["trip_seconds"] / 60
df["hour"]         = df["trip_start_timestamp"].dt.hour
df["weekday"]      = df["trip_start_timestamp"].dt.day_name()  # e.g. 'Monday'
df["date"]         = df["trip_start_timestamp"].dt.date

# ── 2) Build a dict of raw data for each chart ─────────────────────────
chart_data = {}

# HISTOGRAMS
def make_hist(key, series, bins=50):
    arr = series.dropna().values
    if arr.size:
        counts, edges = np.histogram(arr, bins=bins)
        chart_data[key] = {
            "bins":   ((edges[:-1] + edges[1:]) / 2).tolist(),
            "counts": counts.tolist()
        }
    else:
        chart_data[key] = {"bins": [], "counts": []}

make_hist("trip_duration", df["trip_minutes"])
make_hist("trip_distance", df["trip_miles"])
make_hist("fare_distribution", df["fare"])
make_hist("tip_distribution", df["tips"])
make_hist("total_distribution", df["trip_total"])

# LINE CHARTS
#  • Trips by hour
hourly = df.groupby("hour").size()
chart_data["trips_by_hour"] = {
    "x": hourly.index.tolist(),
    "y": hourly.values.tolist()
}

#  • Trips by weekday
weekday_order = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
by_wd = df.groupby("weekday").size().reindex(weekday_order, fill_value=0)
chart_data["trips_by_weekday"] = {
    "x": by_wd.index.tolist(),
    "y": by_wd.values.tolist()
}

#  • Daily trip count (last 30 days)
daily = df.groupby("date").size().sort_index().iloc[-30:]
chart_data["trips_last_30_days"] = {
    "x": [str(d) for d in daily.index],
    "y": daily.values.tolist()
}

# BAR CHARTS
#  • Top 10 pickup areas
top_pu = df["pickup_community_area"].dropna().value_counts().nlargest(10)
chart_data["top_pickup_areas"] = {
    "labels": top_pu.index.astype(str).tolist(),
    "counts": top_pu.values.tolist()
}

#  • Top 10 dropoff areas
top_do = df["dropoff_community_area"].dropna().value_counts().nlargest(10)
chart_data["top_dropoff_areas"] = {
    "labels": top_do.index.astype(str).tolist(),
    "counts": top_do.values.tolist()
}

#  • Top 10 companies by trip count
if "company" in df:
    top_co = df["company"].dropna().value_counts().nlargest(10)
    chart_data["top_companies"] = {
        "labels": top_co.index.tolist(),
        "counts": top_co.values.tolist()
    }

#  • Average fare by pickup area (top 10 busiest)
avg_fare = df.groupby("pickup_community_area")["fare"].mean()
avg_top10 = avg_fare.loc[top_pu.index]
chart_data["avg_fare_by_area"] = {
    "areas":  avg_top10.index.astype(str).tolist(),
    "averages": avg_top10.round(2).tolist()
}

# PIE CHARTS
#  • Payment type breakdown
if "payment_type" in df:
    pay = df["payment_type"].fillna("Unknown").value_counts()
    chart_data["payment_type"] = {
        "labels": pay.index.tolist(),
        "counts": pay.values.tolist()
    }

# SCATTER CHARTS
#  • Distance vs Fare
sc1 = df[["trip_miles","fare"]].dropna()
if len(sc1) > 20000:
    sc1 = sc1.sample(20000, random_state=1)
chart_data["distance_vs_fare"] = sc1.to_dict(orient="records")

#  • Tip vs Distance
sc2 = df[["trip_miles","tips"]].dropna()
if len(sc2) > 20000:
    sc2 = sc2.sample(20000, random_state=1)
chart_data["tips_vs_distance"] = sc2.to_dict(orient="records")

# LOCATION DATA (for map visualizations)
#  • Sample pickup coordinates
coords_pu = df[["pickup_centroid_latitude","pickup_centroid_longitude"]].dropna()
coords_pu = coords_pu.sample(5000, random_state=1) if len(coords_pu) > 5000 else coords_pu
chart_data["pickup_locations"] = coords_pu.rename(
    columns={
      "pickup_centroid_latitude":"lat",
      "pickup_centroid_longitude":"lng"
    }
).to_dict(orient="records")

#  • Sample dropoff coordinates
coords_do = df[["dropoff_centroid_latitude","dropoff_centroid_longitude"]].dropna()
coords_do = coords_do.sample(5000, random_state=1) if len(coords_do) > 5000 else coords_do
chart_data["dropoff_locations"] = coords_do.rename(
    columns={
      "dropoff_centroid_latitude":"lat",
      "dropoff_centroid_longitude":"lng"
    }
).to_dict(orient="records")

# ── 3) Serialize to pickle ───────────────────────────────────────────────
with open("chart_data.pkl", "wb") as f:
    pickle.dump(chart_data, f, protocol=pickle.HIGHEST_PROTOCOL)

print("Serialized chart_data.pkl with charts:", list(chart_data.keys()))


Serialized chart_data.pkl with charts: ['trip_duration', 'trip_distance', 'fare_distribution', 'tip_distribution', 'total_distribution', 'trips_by_hour', 'trips_by_weekday', 'trips_last_30_days', 'top_pickup_areas', 'top_dropoff_areas', 'avg_fare_by_area', 'payment_type', 'distance_vs_fare', 'tips_vs_distance', 'pickup_locations', 'dropoff_locations']
